# Математическая модель метрики "степень галлюцинации"
---
#### Входные данные:
- $C$ = ${c_1, c_2, ..., c_n}$ — список контекстов (строк), где $n$ — количество контекстов.
- $O$ — строка, представляющая вывод модели (ответ).

#### Шаг 1: Извлечение ключевых слов
Для каждой строки $x$ (контекста или вывода модели) определяется множество ключевых слов:
$$
K(x) = \{ w \mid w \in x.split(), w \notin S, len(w) > 3 \}
$$
- $w$ — слово в нижнем регистре.
- $S$ — множество стоп-слов (на русском языке).
- Условие $$w \notin S$$ исключает стоп-слова.
- Условие $len(w) > 3$ исключает короткие слова.

Множества ключевых слов:
- $K_C = \bigcup_{i=1}^n K(c_i)$ — объединение ключевых слов всех контекстов.
- $K_O = K(O)$ — ключевые слова из вывода модели.

#### Шаг 2: Jaccard-подобная схожесть ключевых слов
Схожесть между контекстом и выводом по ключевым словам вычисляется как:
$$
J = \frac{|K_C \cap K_O|}{\max(|K_O|, 1)}
$$
- $|K_C \cap K_O|$ — количество общих ключевых слов.
- $max(|K_O|, 1)$ — нормализация по количеству слов в выводе (избегаем деления на 0).
- Это отражает долю пересечения ключевых слов, избегая чрезмерного штрафа за краткость ответа.

#### Шаг 3: BLEU-оценка
Для каждого контекста $c_i$ вычисляется BLEU-оценка между $c_i$ и $O$:
$$
B_i = BLEU(c_i.split(), O.split(), w)
$$
- $w = (0.7, 0.3, 0, 0)$ — веса для униграмм $(70%)$ и биграмм $(30%)$, триграммы и выше не учитываются.
- Если возникает $ZeroDivisionError,B_i = 0$.

Средняя BLEU-оценка по всем контекстам:
$
B = \frac{1}{n} \sum_{i=1}^n B_i \quad \text{(или 0, если } n = 0\text{)}
$

#### Шаг 4: Штраф за неожиданные слова
Неожиданные слова — это слова в выводе, отсутствующие в контексте:
$
U = K_O \setminus K_C
$
Штраф за неожиданные слова:
$$
P = \begin{cases}
\frac{|U|}{|K_O| + \epsilon} & \text{если } U \neq \emptyset, \\
0 & \text{если } U = \emptyset,
\end{cases}
$$
- $epsilon = 10^{-6}$ — малая константа для избежания деления на 0.
- Штраф пропорционален доле неожиданных слов в выводе.

#### Шаг 5: Итоговый коэффициент
Итоговая метрика "степень галлюцинации" (точнее, степень правдоподобности) вычисляется как:
$
H = \left( 0.6 \cdot B + 0.4 \cdot J \right) \cdot (1 - P)
$
- $0.6 \cdot B$ — вклад BLEU-оценки (60%).
- $0.4 \cdot J$ — вклад пересечения ключевых слов (40%).
- $(1 - P)$ — множитель, уменьшающий итоговую оценку при наличии неожиданных слов.

Ограничение на отрицательные значения:
$
H_{final} = \max(H, 0)
$

---

### Интерпретация
- $H_{final} \in [0, 1]$:
  - $H_{final} \in [0, 0.35]$: Вывод модели полностью соответствует контексту (высокий BLEU, большое пересечение ключевых слов, нет неожиданных слов).
  - $H_{final} \in [0, 0.35]$: Вывод модели сильно отклоняется от контекста (низкий BLEU, мало общих слов, много неожиданных слов).
- Чем выше $H_{final}$, тем меньше "галлюцинаций" в ответе.

In [1]:
!pip install nltk numpy collection typing

In [2]:
import nltk
import numpy as np
from nltk.translate import bleu_score
from nltk.corpus import stopwords
from collections import Counter
from typing import List

nltk.download('stopwords')

def extract_keywords(text: str, stop_words) -> set:
    """Извлекает ключевые слова, исключая стоп-слова и короткие токены."""
    words = text.lower().split()
    return {word for word in words if word not in stop_words and len(word) > 3}

def hallucination_score(contexts: List[str], model_output: str) -> float:
    """Оценивает степень галлюцинации ответа модели по сравнению с контекстом.

    Вычисляет метрику, показывающую, насколько ответ модели соответствует заданным контекстам.
    Использует комбинацию BLEU-оценки, пересечения ключевых слов (Jaccard similarity) и штрафа
    за неожиданные слова. Итоговое значение находится в диапазоне от 0 (полная галлюцинация)
    до 1 (полное соответствие).

    Args:
        contexts (List[str]): Список строк, представляющих контексты для сравнения.
        model_output (str): Ответ модели, который нужно оценить.

    Returns:
        float: Оценка от 0 до 1, где 1 — полное соответствие контексту, 0 — полная галлюцинация.

    Raises:
        ZeroDivisionError: Возможна ошибка деления на ноль в BLEU-оценке, обрабатывается внутри функции.

    Examples:
        >>> contexts = ["Я иду в магазин за хлебом"]
        >>> model_output = "Я иду в магазин за хлебом и молоком"
        >>> hallucination_score(contexts, model_output)
        0.63  # Примерное значение, зависит от BLEU и пересечения
    """

    stop_words = set(stopwords.words('russian'))  # Стоп-слова для русского языка

    # Извлекаем ключевые слова из всех контекстов
    context_keywords = set()
    for context in contexts:
        context_keywords.update(extract_keywords(context, stop_words))

    # Ключевые слова из ответа модели
    output_keywords = extract_keywords(model_output, stop_words)

    # Jaccard similarity (не штрафуем, если ответ короче, но точен)
    keyword_overlap = len(context_keywords & output_keywords) / max(len(output_keywords), 1)

    # BLEU-оценка (с униграммами и биграммами)
    bleu_scores = []
    for context in contexts:
        try:
            score = bleu_score.sentence_bleu(
                references=[context.split()],
                hypothesis=model_output.split(),
                weights=(0.7, 0.3, 0, 0)  # Униграммы (70%) и биграммы (30%)
            )
            bleu_scores.append(score)
        except ZeroDivisionError:
            bleu_scores.append(0)

    bleu_mean = np.mean(bleu_scores) if bleu_scores else 0.0

    # Штраф за неожиданные слова (только если они действительно выбиваются из контекста)
    unexpected_words = output_keywords - context_keywords
    if unexpected_words:
        unexpected_penalty = len(unexpected_words) / (len(output_keywords) + 1e-6)
    else:
        unexpected_penalty = 0  # Если нет неожиданных слов, штрафа нет

    # Итоговый коэффициент (BLEU + семантическое пересечение - штраф за ошибки)
    final_score = (0.6 * bleu_mean + 0.4 * keyword_overlap) * (1 - unexpected_penalty)

    return max(final_score, 0)  # Исключаем отрицательные значения


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Этот код предназначен для оценки степени галлюцинации (т. е. отклонения ответа модели от контекста (создаётся из информации из документов)). Он использует несколько методов анализа текста:

1. Определяет extract_keywords: Извлекает ключевые слова из текста, убирая стоп-слова и короткие слова (меньше 4 символов).
2. Определяет hallucination_score: Оценивает "галлюцинацию" ответа модели относительно контекста.
3. Извлекает ключевые слова: Применяет extract_keywords к контекстам и ответу модели.
Считает пересечение слов: Вычисляет долю общих ключевых слов между контекстом и ответом (Jaccard similarity).
4. Считает BLEU-оценку: Оценивает совпадение униграмм и биграмм между контекстом и ответом (70% и 30% веса).
5. Штрафует за лишние слова: Уменьшает оценку, если в ответе есть слова, отсутствующие в контексте.
Комбинирует метрики: Итоговая оценка = (0.6 * BLEU + 0.4 * пересечение) * (1 - штраф).
6. Возвращает результат: Выдает число от 0 до 1, где 1 — полное соответствие, 0 — полная галлюцинация.

Код комбинирует несколько подходов (BLEU, пересечение ключевых слов, штраф за галлюцинации) для комплексной оценки точности ответа модели. Он полезен для анализа генеративных моделей, где важно минимизировать выдуманные элементы.

In [6]:
# Пример использования

# Пример данных
contexts = [
    "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы, такие как справка по форме 086у или сертификат МОДФ.",
    "Также нужно учесть, что условия предоставления мест и размещения в общежитиях различаются."
     "Более подробную информацию можно получить в Дирекции по управлению общежитиями, гостиницами, учебно-оздоровительными комплексами."
]

model_output_good = "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы"
model_output_bad = "Интернет изобрел Илон Маск в 2010 году в гараже."

# Проверка хорошего ответа
good_score = hallucination_score(contexts, model_output_good)
print(f"Score для хорошего ответа: {good_score:.4f}")

# Проверка плохого (галлюцинирующего) ответа
bad_score = hallucination_score(contexts, model_output_bad)
print(f"Score для плохого ответа: {bad_score:.4f}")

Score для хорошего ответа: 0.4406
Score для плохого ответа: 0.0000


In [4]:
# Number 3.
# Пример 1
model_output_1 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
context_1 = ["Дополнительные документы для продления социальной стипендии нужно предоставить в Центр стипендиальных и благотворительных программ НИУ ВШЭ. Это включает копию заявления и копию действующей справки МСЭ. Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."]
# Проверка хорошего ответа
score = hallucination_score(context_1, model_output_1)
print(f"Score для хорошего ответа: {score:.4f}")

model_output_2 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле SMART LMS, прикрепив отсканированные документы (Паспорт и водительские права)."
# Проверка плохого ответа
score = hallucination_score(context_1, model_output_2)
print(f"Score для хорошего ответа: {score:.4f}")

Score для хорошего ответа: 0.6023
Score для хорошего ответа: 0.3544
